In [1]:
import requests
import pandas as pd
import os

NOTION_DB_ID = '<your database id>'
NOTION_TOKEN = os.environ.get('NOTION_TOKEN')
EODHD_API_TOKEN = os.environ.get('EODHD_API_TOKEN')

In [2]:
def extract_property_value(prop_data):

    prop_type = prop_data.get('type')

    # Handle title properties
    if prop_type == 'title':
        title_array = prop_data.get('title', [])
        if title_array and len(title_array) > 0 and 'plain_text' in title_array[0]:
            return title_array[0]['plain_text']
        return ""

    # Handle rich_text properties
    elif prop_type == 'rich_text':
        rich_text_array = prop_data.get('rich_text', [])
        if rich_text_array and len(rich_text_array) > 0 and 'plain_text' in rich_text_array[0]:
            return rich_text_array[0]['plain_text']
        return ""

    # Handle number properties
    elif prop_type == 'number':
        return prop_data.get('number')

    # Handle select properties
    elif prop_type == 'select':
        select_data = prop_data.get('select')
        if select_data and 'name' in select_data:
            return select_data['name']
        return None

    # Handle formula properties
    elif prop_type == 'formula':
        formula_data = prop_data.get('formula', {})
        formula_type = formula_data.get('type')
        if formula_type:
            return formula_data.get(formula_type)
        return None

    # Handle date properties
    elif prop_type == 'date':
        date_data = prop_data.get('date')
        if date_data and 'start' in date_data:
            return date_data['start']
        return None

    # Handle rollup properties
    elif prop_type == 'rollup':
        rollup_data = prop_data.get('rollup', {})
        rollup_type = rollup_data.get('type')

        # Handle array type rollups (most common)
        if rollup_type == 'array':
            array_data = rollup_data.get('array', [])
            if not array_data:
                return None

            # Get the first item in the array
            first_item = array_data[0]
            first_item_type = first_item.get('type')

            # Handle different types within the rollup array
            if first_item_type == 'select':
                select_data = first_item.get('select')
                if select_data and 'name' in select_data:
                    return select_data['name']
            elif first_item_type == 'rich_text':
                rich_text_array = first_item.get('rich_text', [])
                if rich_text_array and len(rich_text_array) > 0 and 'plain_text' in rich_text_array[0]:
                    return rich_text_array[0]['plain_text']
            elif first_item_type == 'number':
                return first_item.get('number')
            elif first_item_type == 'date':
                date_data = first_item.get('date')
                if date_data and 'start' in date_data:
                    return date_data['start']

        # Handle other rollup types (number, date, etc.)
        elif rollup_type:
            return rollup_data.get(rollup_type)

        return None

    # Handle relation properties
    elif prop_type == 'relation':
        relation_array = prop_data.get('relation', [])
        if relation_array:
            # Return the list of relation IDs
            return [item.get('id') for item in relation_array if 'id' in item]
        return None

    # Handle other property types
    else:
        print(f"Unsupported property type: {prop_type}")
        return None

In [3]:
def get_notion_database(NOTION_DB_ID):
    url = f'https://api.notion.com/v1/databases/{NOTION_DB_ID}/query'
    headers = {
        'Authorization': f'Bearer {NOTION_TOKEN}',
        'Notion-Version': '2021-08-16',
        'Content-Type': 'application/json'
    }

    response = requests.post(url, headers=headers)
    data = response.json()

    # Extract data from the "results" list
    results = data["results"]

    # Create a list to store the extracted data
    notion_db_data = []

    # Extract all properties for each item
    for item in results:
        # Always include the item ID
        stock = {'item_id': item['id']}

        # Dynamically extract all properties
        for prop_name, prop_data in item['properties'].items():
            value = extract_property_value(prop_data)
            # Use the property name as the column name
            stock[prop_name] = value

        notion_db_data.append(stock)

    # Create DataFrame from the extracted data
    df = pd.DataFrame(notion_db_data)
    return df

In [4]:
def update_notion_field(item_id, field_name, field_type, field_value):
    url = f'https://api.notion.com/v1/pages/{item_id}'
    headers = {
        'Authorization': f'Bearer {NOTION_TOKEN}',
        'Notion-Version': '2021-08-16',
        'Content-Type': 'application/json'
    }

    # Create the appropriate property structure based on field type
    if field_type == 'number':
        property_value = {"number": field_value}

    elif field_type == 'title':
        property_value = {
            "title": [
                {
                    "type": "text",
                    "text": {"content": field_value}
                }
            ]
        }

    elif field_type == 'rich_text':
        property_value = {
            "rich_text": [
                {
                    "type": "text",
                    "text": {"content": field_value}
                }
            ]
        }

    elif field_type == 'select':
        property_value = {
            "select": {"name": field_value}
        }

    elif field_type == 'date':
        property_value = {
            "date": {"start": field_value}
        }

    else:
        raise ValueError(f"Unsupported field type: {field_type}")

    # Construct the payload
    payload = {
        "properties": {
            field_name: property_value
        }
    }

    # Send the update request
    response = requests.patch(url, headers=headers, json=payload)
    print(f"Field '{field_name}' updated successfully for item {item_id}.")
    return response.json()

In [6]:
df = get_notion_database(NOTION_DB_ID)
df

,item_id,RSI_,Type,Price,Fast_SMA,RSI,Slow_SMA,Asset Value,Holdings,Asset Name,Trend,DailyChange%,Name
0,20c61905-c09c-8085-a4d1-dc41a1b5076f,🟢 Overbought,,599.140000,595.457800,90.000000,562.820400,5991.400000,10.0,SPDR S&P 500 ETF Trust,✅ Up,0.010269,SPY.US
1,20c61905-c09c-80bd-8c09-c90c3743bebb,🍺 Neutral,,105615.626309,105996.988869,58.392609,102113.760787,10561.562631,0.1,Bitcoin,✅ Up,0.011737,BTC-USD.CC
2,20c61905-c09c-809f-bf63-e9abaae94c89,🍺 Neutral,,203.920000,201.917800,46.873200,203.064400,2039.200000,10.0,Apple Inc,❌ Down,0.016398,AAPL.US
3,20c61905-c09c-800a-ab9f-d6c176ba891e,🔴 Oversold,,174.920000,172.690200,19.000000,162.439600,3498.400000,20.0,Alphabet Inc Class C,✅ Up,0.030092,GOOG.US


In [7]:
# Loop through each stock in the dataframe
for index, row in df.iterrows():

    # Get the ticker symbol from the dataframe
    ticker = row.get('Name')

    # Construct the EODHD API URL
    url = f'https://eodhd.com/api/eod/{ticker}'
    query = {'api_token': EODHD_API_TOKEN, 'fmt': 'json', 'order': 'd'}  # Descending order to get latest first

    # Make the API request
    response = requests.get(url, params=query)

    # Parse the response
    data = response.json()

    # Get the latest price (first entry in descending order)
    latest_data = data[0]
    current_price = latest_data['close']

    # Get the previous day's price (second entry)
    previous_price = data[1]['close'] if len(data) > 1 else None
    daily_change = (current_price - previous_price) / previous_price

    # Update the price and daily change in Notion
    item_id = row['item_id']
    update_notion_field(item_id, 'Price', 'number', current_price)
    update_notion_field(item_id, 'DailyChange%', 'number', daily_change)

Field 'Price' updated successfully for item 20c61905-c09c-8085-a4d1-dc41a1b5076f.
Field 'DailyChange%' updated successfully for item 20c61905-c09c-8085-a4d1-dc41a1b5076f.
Field 'Price' updated successfully for item 20c61905-c09c-80bd-8c09-c90c3743bebb.
Field 'DailyChange%' updated successfully for item 20c61905-c09c-80bd-8c09-c90c3743bebb.
Field 'Price' updated successfully for item 20c61905-c09c-809f-bf63-e9abaae94c89.
Field 'DailyChange%' updated successfully for item 20c61905-c09c-809f-bf63-e9abaae94c89.
Field 'Price' updated successfully for item 20c61905-c09c-800a-ab9f-d6c176ba891e.
Field 'DailyChange%' updated successfully for item 20c61905-c09c-800a-ab9f-d6c176ba891e.


In [8]:
for index, row in df.iterrows():

    ticker = row.get('Name')
    item_id = row['item_id']

    url = f'https://eodhd.com/api/fundamentals/{ticker}'
    query = {'api_token': EODHD_API_TOKEN, 'fmt': 'json'}

    response = requests.get(url, params=query)
    data = response.json()

    # Get the company name from fundamentals data
    if 'General' in data and 'Name' in data['General']:
        company_name = data['General']['Name']
        update_notion_field(item_id, 'Asset Name', 'rich_text', company_name)

        # Get the company name from fundamentals data
    if 'General' in data and 'Type' in data['General']:
        company_name = data['General']['Type']
        update_notion_field(item_id, 'Type', 'rich_text', company_name)

Field 'Asset Name' updated successfully for item 20c61905-c09c-8085-a4d1-dc41a1b5076f.
Field 'Type' updated successfully for item 20c61905-c09c-8085-a4d1-dc41a1b5076f.
Field 'Asset Name' updated successfully for item 20c61905-c09c-80bd-8c09-c90c3743bebb.
Field 'Type' updated successfully for item 20c61905-c09c-80bd-8c09-c90c3743bebb.
Field 'Asset Name' updated successfully for item 20c61905-c09c-809f-bf63-e9abaae94c89.
Field 'Type' updated successfully for item 20c61905-c09c-809f-bf63-e9abaae94c89.
Field 'Asset Name' updated successfully for item 20c61905-c09c-800a-ab9f-d6c176ba891e.
Field 'Type' updated successfully for item 20c61905-c09c-800a-ab9f-d6c176ba891e.


In [9]:
for index, row in df.iterrows():

    ticker = row.get('Name')
    item_id = row['item_id']

    # Get RSI
    url = f'https://eodhd.com/api/technical/{ticker}'
    query = {'api_token': EODHD_API_TOKEN, 'fmt': 'json', 'function': 'rsi', 'order': 'd'}  # Descending order to get latest first

    response = requests.get(url, params=query)

    if response.status_code == 200:
        data = response.json()
        if data and len(data) > 0:
            # Get the latest RSI value (first entry)
            latest_rsi = data[0]['rsi']
            update_notion_field(item_id, 'RSI', 'number', latest_rsi)
            print(f"Updated RSI for {ticker}: {latest_rsi}")

    # Get SMA with period 9
    query = {'api_token': EODHD_API_TOKEN, 'fmt': 'json', 'function': 'sma', 'period': 9, 'order': 'd'}  # Descending order to get latest first

    response = requests.get(url, params=query)

    if response.status_code == 200:
        data = response.json()
        if data and len(data) > 0:
            # Get the latest SMA9 value (first entry)
            latest_sma9 = data[0]['sma']
            update_notion_field(item_id, 'Fast_SMA', 'number', latest_sma9)
            print(f"Updated SMA9 for {ticker}: {latest_sma9}")

    # Get SMA with period 51
    query = {'api_token': EODHD_API_TOKEN, 'fmt': 'json', 'function': 'sma', 'period': 51, 'order': 'd'}  # Descending order to get latest first

    response = requests.get(url, params=query)

    if response.status_code == 200:
        data = response.json()
        if data and len(data) > 0:
            # Get the latest SMA51 value (first entry)
            latest_sma51 = data[0]['sma']
            update_notion_field(item_id, 'Slow_SMA', 'number', latest_sma51)
            print(f"Updated SMA51 for {ticker}: {latest_sma51}")

Field 'RSI' updated successfully for item 20c61905-c09c-8085-a4d1-dc41a1b5076f.
Updated RSI for SPY.US: 55.5532
Field 'Fast_SMA' updated successfully for item 20c61905-c09c-8085-a4d1-dc41a1b5076f.
Updated SMA9 for SPY.US: 598.31
Field 'Slow_SMA' updated successfully for item 20c61905-c09c-8085-a4d1-dc41a1b5076f.
Updated SMA51 for SPY.US: 564.5863
Field 'RSI' updated successfully for item 20c61905-c09c-80bd-8c09-c90c3743bebb.
Updated RSI for BTC-USD.CC: 55.15574411641779
Field 'Fast_SMA' updated successfully for item 20c61905-c09c-80bd-8c09-c90c3743bebb.
Updated SMA9 for BTC-USD.CC: 106363.84164440077
Field 'Slow_SMA' updated successfully for item 20c61905-c09c-80bd-8c09-c90c3743bebb.
Updated SMA51 for BTC-USD.CC: 102773.66040570798
Field 'RSI' updated successfully for item 20c61905-c09c-809f-bf63-e9abaae94c89.
Updated RSI for AAPL.US: 45.9144
Field 'Fast_SMA' updated successfully for item 20c61905-c09c-809f-bf63-e9abaae94c89.
Updated SMA9 for AAPL.US: 201.6044
Field 'Slow_SMA' updated 